1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


hit_rate@k используется, если мы ждем от покупателя разовой покупки такой, как приобретение автомобиля, недвижимости.
precision@k можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно 
являющимися положительными.

В большинстве случаев лучше использовать precision, так как она показывает более полную выборку, чем hit_rate, но при покупке штучных товаров, таких как приобретение автомобиля.

Примеры:
1) Приобретение автомобиля. Покупатель - клиент, автомобиль - товар.

2) Приобретение недвижимости. Покупатель - клиент, недвижимость - товар

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Использование логарифма как функции дисконтирования можно объяснить следующими интуитивными соображениями: с точки зрения ранжирования позиции в начале списка отличаются гораздо сильнее, чем позиции в его конце. Можно использовать степенные функции вместо логарифма
Также логарифм используется в tf idf и logloss

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

MAE (Mean Absolute Error) - Среднее абсолютное отклонение.

RMSE (Root Mean Squared Error) - Корень из среднеквадратичной ошибки.

Precision@k - Метрика Precision, посчитанная на Top-k записях.

Recall@k - Метрика Recall, посчитанная на Top-k записях.

MAP@k - Средний precision по каждому классу

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [1]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [2]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [3]:

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
     
    return precision

In [4]:
precision_at_k(recommended, boughted, k=8)

0.5

In [5]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [6]:
recall_at_k(recommended, boughted, k=8)

0.6666666666666666

In [7]:

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [8]:
ap_k(recommended, boughted, k=8)

0.32

In [9]:
def NDCG_at_k(recommended_list, bought_list, k):
    recommended_list = recommended_list[:k]
    if recommended_list[0] in bought_list:
        dcg = 1
    else: dcg = 0
    for i in range(1, len(recommended_list)):
        if recommended_list[i] in bought_list:
            dcg += 1/np.log(i+1)
       
    idcg = 1
    for i in range(1, k):
        idcg += 1/np.log(i+1)  
    ndcg = dcg / idcg
    
    return ndcg

In [10]:
NDCG_at_k(recommended, boughted, 8)

0.5653142737255068

In [ ]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)